# Combine fits files with same name from n different folders 

#### Algorithms available
 * Add
 * Mean

In [1]:
import os

from IPython.html import widgets
from IPython.display import display

import matplotlib.pyplot as plt

import glob
import pyfits

import widgets_handler

/Users/j35/anaconda/envs/py35/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
root_folder =  "/Volumes/

### Select folder n

#### Select all the folders to add... click CANCEL to let the program know you do not have any more folder to add 

In [5]:
%gui qt

In [6]:
list_folders = []

while(True):
    folder_n = widgets_handler.gui_fname(dir = root_folder)
    if folder_n is '':
        break
    list_folders.append(folder_n)
%matplotlib inline

### Check that all folders have the same number of images

In [7]:
number_of_files = []
list_files_in_folders = []
for _folder in list_folders:
    list_fname = glob.glob(_folder + '/*.fits')
    number_of_files.append(len(list_fname))
    list_files_in_folders.append(list_fname)

assert len(set(number_of_files)) == 1    

## Loading the data 

In [8]:
w = widgets.IntProgress()
w.max = len(list_folders)-1
display(w)

index = 0
data_folders = []

for index_folder, list_folder in enumerate(list_files_in_folders):

    _data_folder = []
    for index_file, filename in enumerate(list_folder):
        hdu_list = pyfits.open(filename)
        hdu = hdu_list[0]
        _data_folder.append(hdu.data)
    
    data_folders.append(_data_folder)
    w.value = index_folder

In [9]:
import numpy as np
np.shape(data_folders)

(3, 1485, 512, 512)

## Retrieving images index

In [10]:
def digit_file_name(full_file_name):
    base_name = os.path.basename(full_file_name)
    [short_name, ext] = base_name.split('.')
    [image_name, image_sequence_number] = short_name.split('_')
    image_name_split = image_name.split('Image')
    image_number = image_name_split[1]
    return [image_name, image_number, image_sequence_number]

In [11]:
list_image_numbers = []
for index_folder, list_folder in enumerate(list_files_in_folders):
    _first_image = list_folder[0]
    [image_name, image_number, image_sequence_number] = digit_file_name(_first_image)
    list_image_numbers.append(image_number)
    
list_image_sequence = []
for index_file, _file in enumerate(list_files_in_folders[0]):
    [image_name, image_number, image_sequence_number] = digit_file_name(_file)
    list_image_sequence.append(image_sequence_number)
    

## Add data 

In [12]:
w = widgets.IntProgress()
number_of_files = len(data_folders[0])
w.max = number_of_files -1
display(w)

sum_data = []
sum_names = []
for index_file in range(number_of_files):

    _add_data = []
    tmp_name = ''
    for index_folder, folder_name in enumerate(list_files_in_folders):
        _list_data = data_folders[index_folder]
        _data = _list_data[index_file]
        [image_name, image_number, image_seqence_number] = digit_file_name(folder_name[index_file])
        if index_folder == 0:
            _add_data = _data
            _tmp_name = "%.2d" %int(image_number)
        else:
            _add_data += _data
            _tmp_name += '_%.2d' %int(image_number)
    
    sum_data.append(_add_data)
    sum_names.append(_tmp_name)
    w.value = index_file

## Exporting data 

In [13]:
from PyQt4 import QtCore, QtGui

def gui_fname(dir=None):
    """Select a folder
    """
    if dir is None: dir ='./'
    dir_name = QtGui.QFileDialog.getExistingDirectory(None, "Select Folder ...",
                                                  dir,
                                                  QtGui.QFileDialog.ShowDirsOnly)
    return dir_name

In [14]:
%gui qt
dir_name = gui_fname(dir = root_folder)
%matplotlib inline

In [15]:
w = widgets.IntProgress()
w.max = len(sum_data)-1
display(w)

for index in range(len(sum_data)):
    
    full_name = os.path.join(dir_name, sum_names[index] + '_' + list_image_sequence[index] + '.fits')

    hdu = pyfits.PrimaryHDU(sum_data[index])
    hdulist = pyfits.HDUList([hdu])
    hdulist.writeto(full_name)
    
    w.value = index